In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [10]:
from tqdm import tqdm
from glob import glob
import pandas as pd
import re
from datasets import Dataset, DatasetDict

In [ ]:
path_train = "SQuAD-UR/train-v2.0"
path_dev = "SQuAD-UR/dev-v2.0"

N_train = 442
N_dev = 35

In [ ]:
def replace_dashes(paragraph):
    paragraph = re.sub(r'(\d)--(\d)', r'\1–\2', paragraph)
    paragraph = paragraph.replace('--', '—')
    return paragraph

In [ ]:
def generateDataset(path, N):
    count = 0
    dataset = {
        "id": [],
        "title": [],
        "context": [],
        "question": [],
        "is_impossible": [],
        "answer": [],
        "answer_start": [],
    }
    for i in tqdm(range(N)):
        df = pd.read_csv(f"{path}/{i}.csv")
        for j in range(len(df)):
            row = df.iloc[j]
            id = row["id"]
            title = row["title"]
            context = row["context"]
            question = row["question"]
            is_impossible = row["is_impossible"]
            review = row["review"]

            context = replace_dashes(context)
            if not review:
                if is_impossible:
                    context = context.replace("••", "")
                    answer_start = -1
                    answer = ""
                else:
                    answer_start = context.index("••")
                    context = context.replace("••", "", 1)
                    answer_end = context.index("••")
                    context = context.replace("••", "", 1)
                    answer = context[answer_start:answer_end]

            dataset["id"].append(id)
            dataset["title"].append(title)
            dataset["context"].append(context)
            dataset["question"].append(question)
            dataset["is_impossible"].append(is_impossible)
            dataset["answer"].append(answer)
            dataset["answer_start"].append(answer_start)

    return Dataset.from_dict(dataset)

In [ ]:
dev = generateDataset(path_dev, N_dev)
train = generateDataset(path_train, N_train)

In [ ]:
dataset = DatasetDict({
    'train': train,
    'validation': dev
})

In [ ]:
dataset.save_to_disk("UQA")